# LDA Classification

### Linear discriminant analysis (LDA; sometimes also called Fisher's linear discriminant) is a linear classifier that projects a p-dimensional feature vector onto a hyperplane that divides the space into two half-spaces. Each half-space represents a class (1 or 0).

# Importing Required Libraries
<ol>
<li>Pandas: To process Datasets</li>
<li>numpy: For fast processing of arrays</li>
<li>Plotly: For Visualization plots</li>
</ol>


In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as pff
import plotly.io as pio
import os
pio.orca.config.executable = os.getcwd()+"/orca"
pio.orca.config.save()

### Function calcualte the mean of each x attribute for a particular class. Function returns the dictionary with class as the key and a vector containg the mean of all x attributes.

In [2]:
def calculate_class_feature_means(train_x, train_y):
    class_feature_means = {}
    for c in classes:
        x = train_x[train_y == c]
        class_feature_means[c] = np.mean(x, axis = 0)
#     print(class_feature_means)
    return class_feature_means

### Function calculate the within class variance scatter matrix this matrix contain the sum of scatter matrix for each class point.

In [3]:
def calculate_within_class_variance(train_x,train_y,class_feature_means):
    within_class_variance = np.zeros((no_of_features,no_of_features))
    for c in classes:
        class_indices = train_y == c
        n = np.count_nonzero(class_indices)
        x = train_x[class_indices, :] - class_feature_means[c][:]
        within_class_variance += (x.T).dot(x)
#     print(within_class_variance.shape)
    return within_class_variance


### Function calculate the between class scatter matrix .

In [4]:
def calculate_between_class_variance(train_x,train_y,class_feature_means):
    between_class_variance = np.zeros((no_of_features,no_of_features))
    whole_mean = np.mean(train_x, axis = 0)
    # print(whole_mean)
    for c in classes:
        class_indices = train_y == c 
        n = np.count_nonzero(class_indices)
    #     print(n)
        x = class_feature_means[c][:]-whole_mean[:]
        between_class_variance += n*np.outer(x,x.T)
#     print(between_class_variance.shape)
    return between_class_variance

### We calculate the projection vector which is the eigen vector corresponding to the max eigen value of inv(within_class_variance)*between_class_variance

In [5]:
def get_projection_vector(within_class_variance, between_class_variance):
    eigen_values, eigen_vectors = np.linalg.eig((np.linalg.pinv(within_class_variance)).dot(between_class_variance))
    # print(eigen_values)
    # print(eigen_vectors)
    maxarg = np.argmax(np.abs(np.real(eigen_values)))
#     print(eigen_values[maxarg])
    projection_vector = np.real(eigen_vectors[:,maxarg])
#     print(projection_vector)
    return projection_vector

### Function plots the point on the projected one dimensional vector

In [9]:
def get_projected_plot(projected_vector,train_y,disease):
    fig = px.scatter(x = projected_vector,y = projected_vector, color=train_y)
    fig.update_layout(
    title = "Projected training points for " + disease +".",
    xaxis_title = "Projection Line",
    yaxis_title = "")
    return fig

### Function fit  normal distribution on the projected point of each class.

In [10]:
def get_normal_dists(projected_vector,train_y):
    normaldist = []
    for c in classes:
#         print(c)
        x = projected_vector[train_y == c]
        mu = np.mean(x)
        sig = np.std(x)
        normaldist.append([mu,sig,c])
    return normaldist


### Gives intersection between two normal distributions.

In [11]:
def find_normal_intersection(m1,m2,std1,std2):
  a = 1/(2*std1**2) - 1/(2*std2**2)
  b = m2/(std2**2) - m1/(std1**2)
  c = m1**2 /(2*std1**2) - m2**2 / (2*std2**2) - np.log(std2/std1)
  return np.roots([a,b,c])

### Function gives intersection point of our normal distributions.

In [12]:
def get_class_intersection(normaldist):
    intersection_point = np.real(find_normal_intersection(normaldist[0][0],normaldist[1][0],normaldist[0][1],normaldist[1][1]))
#     print(intersection_point)
#     print(normaldist)
    # take intersection point lying between two normal curve means.
    intersection = intersection_point[intersection_point > normaldist[0,0]]
    intersection = intersection[intersection < normaldist[1,0]]
    print(intersection)
    return intersection[0]
#     print(intersection)

### Function return class of a given x vector of a point.

In [13]:
def get_class(x,projection_vector,intersection,normaldist):
    projected_point = projection_vector.dot(x)
    if(projected_point > intersection): # if point lies on right side of intersection return right class else left
        return normaldist[1,2]
    else:
        return normaldist[0,2]

### Function measures precision, recall, f-measure and accuracy.

In [14]:
def measure_accuracy(predicted_class, test_y):
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0

    for index in range(len(predicted_class)):
        if(predicted_class[index] == 1 and test_y[index] == 1):
            true_positives += 1
        elif (predicted_class[index] == 1 and test_y[index] == 0):
            false_positives += 1
        elif (predicted_class[index] == 0 and test_y[index] == 1):
            false_negatives += 1
        elif (predicted_class[index] == 0 and test_y[index] == 0):
             true_negatives += 1  
#     print(true_negatives)
#     print(true_positives)
#     print(false_negatives)
#     print(false_positives)
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f_measure = (2 * precision * recall)/(precision + recall)
    print("Total Positives = ",true_positives + false_negatives)
    print("True Posituves = ",true_positives)
    print("Total Negatives = ",true_negatives + false_positives)
    print("True Negatives = ",true_negatives)
    
    print("F measure = ",f_measure)
    print("precison = ", precision)
    print("recall = ", recall)
    accuracy = (true_positives + true_negatives)/(true_positives + true_negatives + false_negatives + false_positives)
    print("Accuracy = ", accuracy )
    print("************************************************************************")
    return [precision, recall, f_measure, accuracy]

### Function calculates and returns the predicted class of data points.

In [15]:
def get_prediction_vector(normaldist, intersection, test_projected):
    predicted_class = []
    # print(test_projected)
    for projected_point in test_projected: 
        if(projected_point > intersection):
            predicted_class.append(normaldist[1,2])
        else:
            predicted_class.append(normaldist[0,2])
    return predicted_class

### Reading the processed dataset

In [16]:
df = pd.read_csv("../Data/Processed/encoded.csv")
df = df.drop(columns = ["district"]) #Not a numerical variable
len(list(df))

149

### These are the list of diseases we are going to do the classification analysis on, the list contain all the diseases that have been diagnosed in our dataset
* There are total of 35 diseases

In [17]:
diseases = ['diagnosed_for_Anaemia',
 'diagnosed_for_Asthma/chronic respiratory failure',
 'diagnosed_for_Blood cancer/leukemia',
 'diagnosed_for_Cancer-Breast',
 'diagnosed_for_Cancer-Gastro-intestinal system',
 'diagnosed_for_Cancer-Genitourinary system',
 'diagnosed_for_Cancer-Respiratory system',
 'diagnosed_for_Cataract',
 'diagnosed_for_Chronic heart disease',
 'diagnosed_for_Chronic heart disease/failure',
 'diagnosed_for_Chronic liver disease',
 'diagnosed_for_Chronic liver failure',
 'diagnosed_for_Chronic renal failure',
 'diagnosed_for_Chronic skin diseases - psoriasis',
 'diagnosed_for_Diabetes',
 'diagnosed_for_Epilepsy',
 'diagnosed_for_Flourosis',
 'diagnosed_for_Gall stone /cholecystitis',
 'diagnosed_for_Glaucoma',
 'diagnosed_for_Goitre/ Thyroid disorder',
 'diagnosed_for_Hypertension',
 'diagnosed_for_Leprosy',
 'diagnosed_for_Myocardial infarction/heart attack',
 'diagnosed_for_Not diagnosed',
 'diagnosed_for_Others(Hernia;Hydrocele;Peptic Ulcer etc)',
 'diagnosed_for_Piles/Anal fissure & Anal fistula',
 'diagnosed_for_Pyorrhea',
 'diagnosed_for_Renal stone',
 'diagnosed_for_Rheumatic fever/Rheumatic heart disease',
 'diagnosed_for_Rheumatoid arthritis / osteoarthritis',
 'diagnosed_for_Sinusitis,Tonsilitis',
 'diagnosed_for_Skin Cancer',
 'diagnosed_for_Stroke/Cerebrovascular accident',
 'diagnosed_for_Tuberculosis',
 'diagnosed_for_Tumour(any type)']
len(list(diseases))

35

* We remove the diseases columns in our dataset call that dataframe classifiers as they are going to classify if our sample point should be classified positive or negative. 
* All the diseases data is in y_data

In [18]:
classifiers = df.drop(columns = diseases)
y_data = df[diseases]

### Splitting into train and test dataset
* We split the dataset into train an test in 80:20 which is the most genral used approach 
* Also, initialising some variables which will be used in our code.

In [19]:
data_size = len(df)
train_size = (int)(0.80 * data_size)
data_array = classifiers.to_numpy()
y_array = y_data.to_numpy()
no_of_features = data_array.shape[1]
classes = [0,1]
train_x = data_array[0:train_size, :]
test_x = data_array[train_size:,:]
accuracy_lists = []

In [20]:
# deleting to free some space
del(df)
del(y_data)

### We do LDA analysis on each disease calcuate the accuracy of the classification.
* We first train our model on the train dataset and get the projection vector to project the n dimensional x vector to 1 dimensional.
* The projection is done in such a way that there is maximum difference between the mean of two class points and minimum distace between points within a class.
* The two classes are than seperated by fitting a normal distribution of two classes and taking the intersection of the two normal distribution as the seperator

In [21]:

for index in range(len(diseases)):
    #We take the y vector for particular disease.
    train_y = y_array[0:train_size, index]
    test_y = y_array[train_size:, index]
    
    # We calcualte the mean of each x attribute for a particular class. Function returns the dictionary
    # with class as the key and a vector containg the mean of all x attributes
    class_feature_means = calculate_class_feature_means(train_x,train_y)
    
    # We calculate the within class variance scatter matrix this matrix contain the sum of scatter matrix
    # for each class points 
    within_class_variance = calculate_within_class_variance(train_x,train_y, class_feature_means)
    
    # We calculate the sum of scatter matrix of within class means.
    between_class_variance = calculate_between_class_variance(train_x,train_y,class_feature_means)
    
    # We calculate the projection vector which is the eigen vector corresponding to the max eigen value of
    # inv(within_class_variance)*between_class_variance
    projection_vector = get_projection_vector(within_class_variance, between_class_variance)
#     print(projection_vector.shape())
    
    # Project the 114 dimension x_vector to 1 dimension
    projected_vector = train_x.dot(projection_vector)
#     print(projected_vector.shape)
#     print(train_y.shape)
    fig = get_projected_plot(projected_vector,train_y, diseases[index])
    fig.update_layout(
    width=1600,
    height=900
    )
    pio.write_image(fig,"../Plots/Classification/Projection Disease " +str(index + 1)+".png",format='png')
    # We fit a normal distribution on the projected point of each class and find the intersection of these two
    # normal distribution and this intersection distinguish these two class.
    
    normaldist = get_normal_dists(projected_vector,train_y)
    normaldist = np.array(normaldist)
    normaldist = normaldist[np.argsort(normaldist[:,0]),:]
    intersection = get_class_intersection(normaldist)
#     create_dist_plot(projected_vector, train_y)

    # Now we check our model on the test data by projecting data on 1 dimension and comparing the 
    # predicted class with actual class
    test_projected = test_x.dot(projection_vector)
    predicted_class = get_prediction_vector(normaldist, intersection, test_projected)
    print(diseases[index])      
    accuracy_measures_values = measure_accuracy(predicted_class, test_y)
    accuracy_measures_values.append(diseases[index])
    accuracy_lists.append(accuracy_measures_values)

    

[-0.06244525]
diagnosed_for_Anaemia
Total Positives =  174
True Posituves =  39
Total Negatives =  219205
True Negatives =  212378
F measure =  0.011079545454545455
precison =  0.005680163122633266
recall =  0.22413793103448276
Accuracy =  0.96826496610888
************************************************************************
[0.25501134]
diagnosed_for_Asthma/chronic respiratory failure
Total Positives =  2706
True Posituves =  1041
Total Negatives =  216673
True Negatives =  215215
F measure =  0.39999999999999997
precison =  0.41656662665066024
recall =  0.38470066518847007
Accuracy =  0.9857643621312888
************************************************************************
[-0.03578791]
diagnosed_for_Blood cancer/leukemia
Total Positives =  57
True Posituves =  26
Total Negatives =  219322
True Negatives =  217520
F measure =  0.027586206896551724
precison =  0.014223194748358862
recall =  0.45614035087719296
Accuracy =  0.9916445967936767
***************************************

Total Positives =  4605
True Posituves =  3498
Total Negatives =  214774
True Negatives =  203896
F measure =  0.36857910542121075
precison =  0.24332220367278798
recall =  0.7596091205211727
Accuracy =  0.9453685174971168
************************************************************************
[0.10129467]
diagnosed_for_Piles/Anal fissure & Anal fistula
Total Positives =  958
True Posituves =  162
Total Negatives =  218421
True Negatives =  217128
F measure =  0.13427268959801078
precison =  0.11134020618556702
recall =  0.16910229645093947
Accuracy =  0.9904776665040865
************************************************************************
[-0.00820069]
diagnosed_for_Pyorrhea
Total Positives =  123
True Posituves =  48
Total Negatives =  219256
True Negatives =  219033
F measure =  0.2436548223350254
precison =  0.17712177121771217
recall =  0.3902439024390244
Accuracy =  0.998641620209774
************************************************************************
[-0.08392893]
diagno

In [22]:
accuracy_matrix = pd.DataFrame(accuracy_lists, columns = ["precision", "recall", "f_measure", "accuracy", "disease"])
accuracy_matrix.to_csv("../Data/Results/accuracy.csv")

In [23]:
print(accuracy_matrix)

    precision    recall  f_measure  accuracy  \
0    0.005680  0.224138   0.011080  0.968265   
1    0.416567  0.384701   0.400000  0.985764   
2    0.014223  0.456140   0.027586  0.991645   
3    0.001018  0.444444   0.002032  0.946276   
4    0.075956  0.526132   0.132747  0.991006   
5    0.080139  0.388514   0.132871  0.993158   
6    0.002523  0.459016   0.005019  0.949398   
7    0.222368  0.637736   0.329756  0.996868   
8    0.210406  0.400000   0.275759  0.991189   
9    0.065139  0.739726   0.119734  0.992761   
10   0.009323  0.175772   0.017708  0.962576   
11   0.003992  0.377778   0.007900  0.980536   
12   0.000618  0.736842   0.001236  0.896827   
13   0.322581  0.477273   0.384968  0.996941   
14   0.048621  0.660751   0.090577  0.911395   
15   0.038462  0.172932   0.062927  0.990633   
16   0.099806  0.415323   0.160937  0.995104   
17   0.011817  0.296296   0.022727  0.990592   
18   0.050000  0.808511   0.094176  0.996668   
19   0.023364  0.062305   0.033985  0.99